In [7]:
import pandas as pd   # основная библиотека для работы с таблицами
import numpy as np    # библиотека для численных вычислений
 
# Эти две библиотеки не обязательны, но полезны для визуализации выбросов
import matplotlib.pyplot as plt
import seaborn as sns
 
 
# ================================================
# НАСТРОЙКИ: КАКИЕ СТОЛБЦЫ БУДЕМ ИЗУЧАТЬ
# ================================================
 
# Здесь мы явно указываем, какие 8 колонок берём для анализа.
# ❗ ЕСЛИ у тебя в df другие названия колонок,
#    просто поменяй элементы списков ниже.
 
numeric_cols = [
    "rectal_temp",         # ректальная температура (число)
    "pulse",               # пульс (число)
    "respiratory_rate",    # частота дыхания (число)
    "packed_cell_volume"   # гематокрит (число)
]
 
categorical_cols = [
    "pain",                # уровень боли (категориальная)
    "abdominal_distension",# вздутие живота (категориальная)
    "outcome",             # исход (категориальная)
    "surgery"              # была ли операция (категориальная)
]
 
selected_cols = numeric_cols + categorical_cols
 
 
# ================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ================================================
 
def select_columns(df: pd.DataFrame,
                   cols: list[str]) -> pd.DataFrame:
    """
    Функция выбирает из исходного датафрейма только нужные столбцы.
 
    df   - исходный датафрейм со всеми данными
    cols - список названий столбцов, которые нужны для анализа
 
    Возвращает новый датафрейм только с этими столбцами.
    """
    # .copy() — делаем копию, чтобы не портить исходный df
    df_sel = df[cols].copy()
    return df_sel
 
 
def basic_stats_numeric(df: pd.DataFrame,
                        numeric_cols: list[str]) -> pd.DataFrame:
    """
    Считает базовые статистики для ЧИСЛОВЫХ столбцов.
 
    Для каждого столбца получаем:
    - count (сколько наблюдений, не NaN)
    - mean (среднее)
    - std (стандартное отклонение)
    - min, max
    - 25%, 50% (медиана), 75% квартили
    - моду (mode)
 
    Возвращает датафрейм с описанием (describe),
    а моду печатает отдельно.
    """
    print("=== Базовые статистики для числовых столбцов ===\n")
 
    # describe() сразу считает основные статистики для всех числовых колонок
    desc = df[numeric_cols].describe().T  # .T — транспонируем, чтобы столбцы были строками
    print(desc, "\n")
 
    # Считаем моду (наиболее часто встречающееся значение) по каждому столбцу
    print("Моды для числовых столбцов:")
    for col in numeric_cols:
        mode_series = df[col].mode(dropna=True)
        if mode_series.empty:
            print(f"  {col}: мода не определена (нет данных)")
        else:
            # mode() может вернуть несколько значений, возьмём первое
            print(f"  {col}: мода = {mode_series.iloc[0]}")
    print()
 
    return desc
 
 
def basic_stats_categorical(df: pd.DataFrame,
                            categorical_cols: list[str]) -> None:
    """
    Считает базовые статистики для КАТЕГОРИАЛЬНЫХ столбцов.
 
    Для каждой категории:
    - выводим количество различных значений (nunique)
    - выводим частоты значений (value_counts)
    - находим моду (самое частое значение)
    """
    print("=== Базовые статистики для категориальных столбцов ===\n")
 
    for col in categorical_cols:
        print(f"Столбец: {col}")
        print(f"  Количество уникальных значений: {df[col].nunique(dropna=True)}")
 
        # value_counts показывает, сколько раз встречается каждое значение
        print("  Частоты значений (включая NaN):")
        print(df[col].value_counts(dropna=False))
 
        # Мода
        mode_series = df[col].mode(dropna=True)
        if mode_series.empty:
            print("  Мода: отсутствует (нет данных)")
        else:
            print(f"  Мода: {mode_series.iloc[0]}")
        print("-" * 40)
 
 
def find_outliers_iqr(series: pd.Series) -> tuple[pd.Series, float, float]:
    """
    Ищем выбросы в одном числовом столбце с помощью правила IQR.
 
    IQR (interquartile range) = Q3 - Q1
    Нижняя граница = Q1 - 1.5 * IQR
    Верхняя граница = Q3 + 1.5 * IQR
 
    Всё, что < нижней границы или > верхней границы, считаем выбросами.
 
    Возвращаем:
    - булеву маску (True, где выброс),
    - нижнюю границу,
    - верхнюю границу.
    """
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
 
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
 
    # mask = True там, где значение < lower_bound или > upper_bound
    mask = (series < lower_bound) | (series > upper_bound)
 
    return mask, lower_bound, upper_bound
 
 
def analyze_outliers(df: pd.DataFrame,
                     numeric_cols: list[str]) -> pd.DataFrame:
    """
    Для каждого числового столбца:
    - находим выбросы по правилу IQR
    - считаем их количество
    - выводим границы
 
    Возвращаем датафрейм с суммарной информацией по выбросам:
    столбец, количество выбросов, доля выбросов.
    """
    print("=== Анализ выбросов (IQR-метод) для числовых столбцов ===\n")
 
    rows = []
 
    for col in numeric_cols:
        series = df[col].dropna()  # игнорируем пропуски при поиске выбросов
        mask, lower, upper = find_outliers_iqr(series)
 
        n_outliers = mask.sum()  # сколько True
        n_total = series.shape[0]
        share = n_outliers / n_total if n_total > 0 else 0
 
        print(f"Столбец: {col}")
        print(f"  Нижняя граница: {lower:.3f}")
        print(f"  Верхняя граница: {upper:.3f}")
        print(f"  Количество выбросов: {n_outliers} из {n_total} ({share:.2%})")
 
        # Можно при желании посмотреть сами выбросы:
        # print(series[mask])
 
        print("-" * 40)
 
        rows.append({
            "column": col,
            "n_outliers": n_outliers,
            "n_non_null": n_total,
            "share_outliers": share
        })
 
    # Делаем из списка словарей итоговый датафрейм
    outliers_summary = pd.DataFrame(rows)
    return outliers_summary
 
 
def plot_outliers_boxplots(df: pd.DataFrame,
                           numeric_cols: list[str]) -> None:
    """
    Дополнительная функция: рисуем boxplot'ы для числовых столбцов,
    чтобы визуально увидеть выбросы.
 
    Это НЕ обязательно для ДЗ, но полезно для понимания.
    """
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
 
    # Перестраиваем данные в "длинный" формат для удобства рисования
    df_long = df[numeric_cols].melt(var_name="column", value_name="value")
 
    sns.boxplot(data=df_long, x="column", y="value")
    plt.title("Boxplot для числовых столбцов (выбросы видны как точки вне усов)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
 
 
def missing_values_report(df: pd.DataFrame,
                          cols: list[str]) -> pd.DataFrame:
    """
    Считает количество пропусков (NaN) по выбранным столбцам.
 
    Возвращает датафрейм с:
    - column      — имя столбца
    - n_missing   — сколько пропусков
    - share_missing — доля пропусков
    """
    print("=== Отчёт по пропускам (NaN) ===\n")
 
    rows = []
 
    for col in cols:
        n_missing = df[col].isna().sum()
        n_total = df[col].shape[0]
        share = n_missing / n_total if n_total > 0 else 0
 
        print(f"{col}: пропусков {n_missing} из {n_total} ({share:.2%})")
 
        rows.append({
            "column": col,
            "n_missing": n_missing,
            "n_total": n_total,
            "share_missing": share
        })
 
    print()
    return pd.DataFrame(rows)
 
 
def fill_missing_values(df: pd.DataFrame,
                        numeric_cols: list[str],
                        categorical_cols: list[str]) -> pd.DataFrame:
    """
    Создаёт НОВЫЙ датафрейм, где НЕТ пропусков.
 
    Принята следующая стратегия (её потом можно описать в отчёте):
    - для числовых столбцов: заполняем пропуски МЕДИАНОЙ
      (устойчива к выбросам, хороший дефолтный выбор)
    - для категориальных столбцов: заполняем пропуски МОДОЙ
      (самая частая категория)
 
    Возвращает df_filled — копию df без пропусков.
    """
    df_filled = df.copy()
 
    # Числовые
    for col in numeric_cols:
        median_value = df_filled[col].median()
        print(f"Заполняем пропуски в числовом столбце {col} медианой = {median_value}")
        df_filled[col] = df_filled[col].fillna(median_value)
 
    # Категориальные
    for col in categorical_cols:
        mode_series = df_filled[col].mode(dropna=True)
        if mode_series.empty:
            # если вдруг все значения NaN — можно придумать спец. категорию
            fill_value = "Unknown"
            print(f"Все значения в {col} пропущены, заполняем 'Unknown'")
        else:
            fill_value = mode_series.iloc[0]
            print(f"Заполняем пропуски в категориальном столбце {col} модой = {fill_value}")
 
        df_filled[col] = df_filled[col].fillna(fill_value)
 
    print("\nВсе пропуски заполнены.")
    return df_filled
 
 
# ================================================
# ОСНОВНОЙ "ОРКЕСТРАТОР" ДЛЯ ДЗ
# ================================================
 
def run_homework_analysis(df: pd.DataFrame) -> None:
    """
    Основная функция, которая запускает все шаги домашнего задания:
 
    1) Базовое изучение: выбор 8 колонок, расчёт базовых метрик.
    2) Работа с выбросами: поиск выбросов в числовых столбцах.
    3) Работа с пропусками: подсчёт пропусков, заполнение и получение df без NaN.
 
    df — датафрейм со всеми исходными данными (из ноутбука).
    """
 
    # ---------- Задание 1. Базовое изучение ----------
    print("\n========== ЗАДАНИЕ 1: БАЗОВОЕ ИЗУЧЕНИЕ ==========\n")
 
    df_sel = select_columns(df, selected_cols)
 
    # Базовые статистики для числовых
    desc_numeric = basic_stats_numeric(df_sel, numeric_cols)
 
    # Базовые статистики для категориальных
    basic_stats_categorical(df_sel, categorical_cols)
 
    # Здесь, в отдельной markdown-ячейке в ноутбуке,
    # можно кратко описать результаты: диапазоны, медианы,
    # какие значения категорий чаще встречаются и т.д.
 
    # ---------- Задание 2. Работа с выбросами ----------
    print("\n========== ЗАДАНИЕ 2: РАБОТА С ВЫБРОСАМИ ==========\n")
 
    outliers_summary = analyze_outliers(df_sel, numeric_cols)
 
    print("\nСводная таблица по выбросам:")
    print(outliers_summary)
 
    # Дополнительно можно визуализировать выбросы
    # (по желанию, не обязательно для сдачи ДЗ):
    # plot_outliers_boxplots(df_sel, numeric_cols)
 
    # В отдельной markdown-ячейке:
    # - сформулировать гипотезы, откуда взялись выбросы
    #   (ошибки ввода, редкие тяжёлые состояния и т.п.)
    # - принять решение: оставляем, обрезаем, заменяем и т.д.
    # В этом коде мы НИЧЕГО с выбросами не делаем автоматически,
    # только считаем и показываем.
 
    # ---------- Задание 3. Работа с пропусками ----------
    print("\n========== ЗАДАНИЕ 3: РАБОТА С ПРОПУСКАМИ ==========\n")
 
    # 1) Считаем пропуски
    missing_report = missing_values_report(df_sel, selected_cols)
    print("Таблица с пропусками:")
    print(missing_report, "\n")
 
    # 2) Заполняем пропуски по выбранной стратегии (медиана / мода)
    df_no_missing = fill_missing_values(df_sel, numeric_cols, categorical_cols)
 
    # Проверим, что пропусков действительно нет
    print("\nПроверяем, остались ли пропуски:")
    print(df_no_missing.isna().sum())
 
    # Теперь df_no_missing — датафрейм БЕЗ пропусков,
    # его можно использовать дальше для моделей и т.п.
    # При желании можно вернуть его из функции:
    # return df_no_missing
 
 
# ================================================
# ПРИМЕР ЗАПУСКА В НОУТБУКЕ
# ================================================
 
# Предполагаем, что в Базовые_Понятния.ipynb у тебя уже есть df.
# Тогда достаточно вызвать:
#
run_homework_analysis(df)
#
# Если df пока нет, можно загрузить свои данные, например так:
#
# df = pd.read_csv("horse-colic.csv")  # тут заменить на свой путь и параметры
# run_homework_analysis(df)


========== ЗАДАНИЕ 1: БАЗОВОЕ ИЗУЧЕНИЕ ==========



KeyError: "None of [Index(['rectal_temp', 'pulse', 'respiratory_rate', 'packed_cell_volume',\n       'pain', 'abdominal_distension', 'outcome', 'surgery'],\n      dtype='object')] are in the [columns]"

In [1]:
import pandas as pd
import numpy as np
 
df = pd.read_csv("horse_data.csv", header=None, na_values='?')
print(df.shape)
print()
 
 
# Задание 1. Базовое изучение
# 1.1. Выбор 8 столбцов
# Я выберу 8 признаков, чтобы были и числовые, и категориальные, и при этом они были клинически осмысленны.
# Числовые:
# 3 — rectal temperature (ректальная температура, °C)
# 4 — pulse (пульс, уд/мин)
# 5 — respiratory rate (частота дыхания, вдохов/мин)
# 18 — packed cell volume (PCV, гематокрит, %)
# Категориальные:
# 0 — surgery? (1 — была операция, 2 — лечили без операции)
# 1 — age (1 — взрослый, 2 — молодой; значение 9 — подозрительное, обсудим ниже)
# 12 — abdominal distension (раздутие живота, 1–4)
# 22 — outcome (исход: 1 — выжил, 2 — умер, 3 — эвтаназия)
cols = [0, 1, 3, 4, 5, 12, 18, 22]
df_sub = df[cols]
print(df_sub.head())
print(df_sub.describe().T)
#Числовые признаки
#Ректальная температура (столбец 3)
#count = 240 (есть пропуски)
#mean ≈ 38.17 °C
#median = 38.2 °C
#min = 35.4, max = 40.8
#Q1 = 37.8, Q3 = 38.5
#Нормальная температура по описанию ≈ 37.8 °C.
#→ Большинство лошадей вблизи нормы или чуть выше, но есть и пониженные (шок) и повышенные значения (инфекция/воспаление).
#Пульс (столбец 4)
#count = 276
#mean ≈ 71.9 уд/мин
#median = 64
#min = 30, max = 184
#Q1 = 48, Q3 = 88
#Норма для взрослых: 30–40.
#→ Средний пульс сильно выше нормы, что ожидаемо при болевом синдроме и шоке. Есть очень высокие значения (150+).
#Частота дыхания (столбец 5)
#count = 242
#mean ≈ 30.4 вдохов/мин
#median = 24.5
#min = 8, max = 96
#Q1 = 18.5, Q3 = 36
#Норма: 8–10.
#→ У большинства дыхание учащено, что соответствует боли, стрессу, нарушению циркуляции.
#PCV — объём эритроцитов (столбец 18)
#count = 271
#mean ≈ 46.3 %
#median = 45 %
#min = 23, max = 75
#Q1 = 38, Q3 = 52
#Норма: 30–50.
#→ Медиана ближе к верхней границе, у многих животных признаки сгущения крови (дегидратация, шок).
 
for c in [0, 1, 12, 22]:
    print("Столбец", c)
    print(df[c].value_counts(dropna=False))
    print()
 
#Результаты:
#0 — surgery?
#1 (была операция): 180
#2 (без операции): 119
#NaN: 1
#→ Чуть больше половины случаев оперированы.
#1 — age
#1 (adult): 276
#9: 24
#По описанию допустимы только 1 и 2. Значение 9 — некорректный код, его будем считать «пропуском» (ошибкой в данных).
#12 — abdominal distension
#1 (none): 76
#2 (slight): 65
#3 (moderate): 65
#4 (severe): 38
#NaN: 56
#→ У значимой части лошадей есть раздувание живота, что подтверждает тяжесть состояния.
#22 — outcome
#1 (lived): 178
#2 (died): 77
#3 (euthanized): 44
#NaN: 1
#→ Примерно 60% выжили, 40% умерли или были эвтаназированы.
 
#Задание 2. Работа с выбросами
#Ищем выбросы в числовых столбцах 3, 4, 5, 18 по правилу Tukey (IQR):
#IQR = Q3 − Q1
#нижняя граница = Q1 − 1.5·IQR
#верхняя граница = Q3 + 1.5·IQR
 
num_cols = [3, 4, 5, 18]
outliers_info = {}
 
for c in num_cols:
    s = df[c].dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = s[(s < lower) | (s > upper)]
    outliers_info[c] = {
        "count": len(s),
        "n_outliers": len(outliers),
        "q1": q1,
        "q3": q3,
        "lower": lower,
        "upper": upper,
        "min": s.min(),
        "max": s.max(),
        "out_min": outliers.min() if len(outliers) > 0 else None,
        "out_max": outliers.max() if len(outliers) > 0 else None,
    }
 
print(outliers_info,)
 
 
# Кратко:
# Температура (3)
# Q1 = 37.8, Q3 = 38.5, IQR ≈ 0.7
# Диапазон без выбросов: [36.75; 39.55]
# min = 35.4, max = 40.8
# выбросов: 14 (от 35.4 до 40.8)
# Интерпретация:
# Низкие (~35.4) — выраженный шок.
# Высокие (~40+) — сильная лихорадка.
# Клинически такие значения возможны и важны, это не «ошибки ввода».
# Пульс (4)
# Q1 = 48, Q3 = 88, IQR = 40
# Диапазон без выбросов: [−12; 148]
# min = 30, max = 184
# выбросов: 5 (150–184)
# Норма 30–40. Пульс 150–180 — тяжёлый шок/боль, но физиологически достижим.
# → Выбросы отражают самые тяжёлые случаи.
# Частота дыхания (5)
# Q1 = 18.5, Q3 = 36, IQR = 17.5
# Диапазон без выбросов: [−7.75; 62.25]
# min = 8, max = 96
# выбросов: 17 (66–96)
# Норма 8–10. Значения 60–90 — тяжёлая одышка. Полностью укладывается в клинику.
# PCV (18)
# Q1 = 38, Q3 = 52, IQR = 14
# Диапазон без выбросов: [17; 73]
# min = 23, max = 75
# выбросов: 3 (74–75)
# Гематокрит 74–75% — сильная дегидратация/компрометация кровообращения. Возможное, пусть и редкое состояние.
# Решение по выбросам
# Все выявленные «выбросы» находятся в реалистичных физиологических диапазонах.
# С учётом медицинского контекста это важные тяжёлые случаи, а не шум.
# 👉 Решение:
# Не удалять и не обрезать выбросы.
# Мы:
# оставляем их в данных,
# в дальнейших моделях при необходимости используем робастные методы (медиана, IQR, робастные оценки) или модели, устойчивые к хвостам распределения.
 
 
 
 
# Задание 3. Работа с пропусками
print(df_sub.isna().sum())
 
mode0 = df[0].mode().iloc[0]  # 1.0
df_clean = df.copy()
df_clean.loc[df_clean[1] == 9, 1] = np.nan
mode1 = df_clean[1].mode().iloc[0]  # скорее всего 1.0
df_clean[1] = df_clean[1].fillna(mode1)
df[[12, 22]].groupby(22)[12].value_counts(dropna=False)
 
def fill_cat_group_mode(col, by):
    g = df_clean.groupby(by)[col]
    def repl(x):
        mode_vals = x.mode()
        if len(mode_vals) == 0:
            return x
        return x.fillna(mode_vals.iloc[0])
    df_clean[col] = g.transform(repl)
 
fill_cat_group_mode(12, 22)
mode22 = df_clean[22].mode().iloc[0]
df_clean[22] = df_clean[22].fillna(mode22)
 
df[[3, 4, 5, 18, 22]].groupby(22).median()
 
 
num_cols = [3, 4, 5, 18]
 
for c in num_cols:
    # сначала заполняем медианой внутри исхода
    df_clean[c] = df_clean.groupby(22)[c].transform(
        lambda s: s.fillna(s.median())
    )
    # на всякий случай, если в какой-то группе всё было NaN — добиваем общей медианой
    df_clean[c] = df_clean[c].fillna(df_clean[c].median())
 
df_clean[cols].isna().sum()

(300, 28)

    0   1     3      4     5    12    18   22
0  2.0   1  38.5   66.0  28.0  4.0  45.0  2.0
1  1.0   1  39.2   88.0  20.0  2.0  50.0  3.0
2  2.0   1  38.3   40.0  24.0  1.0  33.0  1.0
3  1.0   9  39.1  164.0  84.0  4.0  48.0  2.0
4  2.0   1  37.3  104.0  35.0  NaN  74.0  2.0
    count       mean        std   min   25%   50%   75%    max
0   299.0   1.397993   0.490305   1.0   1.0   1.0   2.0    2.0
1   300.0   1.640000   2.173972   1.0   1.0   1.0   1.0    9.0
3   240.0  38.167917   0.732289  35.4  37.8  38.2  38.5   40.8
4   276.0  71.913043  28.630557  30.0  48.0  64.0  88.0  184.0
5   242.0  30.417355  17.642231   8.0  18.5  24.5  36.0   96.0
12  244.0   2.266393   1.065131   1.0   1.0   2.0   3.0    4.0
18  271.0  46.295203  10.419335  23.0  38.0  45.0  52.0   75.0
22  299.0   1.551839   0.737187   1.0   1.0   1.0   2.0    3.0
Столбец 0
0
1.0    180
2.0    119
NaN      1
Name: count, dtype: int64

Столбец 1
1
1    276
9     24
Name: count, dtype: int64

Столбец 12
12
1.0

0     1
1     0
3     0
4     0
5     0
12    1
18    0
22    0
dtype: int64